In [2]:
import pandas as pd

# Read the CSV of Value of Energy Cost Savings and check the head of the dataframe
df = pd.read_csv("Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020.csv", parse_dates=True)
df.head()

,Company Name,company contact,company email,company phone,Address,City,State,Postcode,Industry,Industry descr,...,Savings from beginning receiving benefits,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1 State Street,David Neiburg,eskoblo@energyspec.com,7186779077,1114 Avenue J,Brooklyn,NY,11230,Commercial,35 story building,...,349.78,Brooklyn,40.624784,-73.964920,314.0,44.0,534.0,3179818.0,3.067130e+09,Midwood
1,100 Church LLC,Edward A. Carroll,NaN,2125663000,100 Church Street,NY,NY,10007,Commercial,21 Floors,...,4423698.05,New York,40.712968,-74.009490,101.0,1.0,21.0,1001409.0,1.001250e+09,SoHo-TriBeCa-Civic Center-Little Italy
2,100 Wall Investments,Jerry Branam,NaN,2122487160,96 Wall Street,New York,NY,10005,Landlord,NaN,...,1814542.62,New York,40.705052,-74.007199,101.0,1.0,7.0,1000875.0,1.000380e+09,Battery Park City-Lower Manhattan
3,1-10 Bush Terminal Owner LP,Andrew Kimball,akimball@industrycity.com,3474178429,"220 36th St, Suite 2A",Brooklyn,NY,11232,Landlord,Special Eligible Premises for eligible businesses,...,6755.31,Brooklyn,40.656061,-74.007334,307.0,38.0,2.0,3398687.0,3.006950e+09,Sunset Park West
4,"120 Broadway Acquisitions JV, LLC (Ldmk)",Carl Letich,clettich@silvprop.com,2124064020,120 Broadway,New York,NY,10271,Commercial,NaN,...,3293055.08,New York,40.708546,-74.011041,101.0,1.0,7.0,1001026.0,1.000478e+09,Battery Park City-Lower Manhattan


In [ ]:
# 1. How many companies are listed in the file?
len(pd.unique(df["Company Name"]))

In [ ]:
# 2. What is the total number of jobs created for businesses in Queens?
df[df['Borough']=='Queens']['Job created'].sum()

In [ ]:
# 3. How many different unique email domains names are there in the data set?
len(pd.unique(df['company email'].str.split('@').str[-1]))

In [ ]:
# 4. Considering only NTAs with at least 5 listed businesses, what is the average total savings and the total jobs created for each NTA?
NTA_over_5 = df[df['NTA'].map(df['NTA'].value_counts())>5].groupby(['NTA'],as_index=False).agg({'Total Savings': ['mean'], 'Job created': ['count']})
NTA_over_5.columns = NTA_over_5.columns.get_level_values(0)
NTA_over_5.columns = ['NTA','Avg Total Savings','Jobs created']
print(NTA_over_5)

In [ ]:
# 5. Save result as CSV
NTA_over_5.to_csv("Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020_NAT_over_5.csv")

In [ ]:
# Day 5: Plotting in MatPlotLib
from calendar import month
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns

# Set up standard scale scatter plot and show
plt.scatter(NTA_over_5['Avg Total Savings'],NTA_over_5['Jobs created'])
plt.xlabel('Avg Total Savings')
plt.ylabel('Jobs Created')
plt.title('Scatter Plot - Standard Scale')
plt.show()

# Set up logarithmic scale scatter plot and show
print("Scatter Plot - Standard Scale: ")
NTA_over_5['Log Avg Total Savings']=np.log(NTA_over_5['Avg Total Savings'])
plt.scatter(NTA_over_5['Log Avg Total Savings'],NTA_over_5['Jobs created'])
plt.xlabel('Total Savings')
plt.ylabel('Jobs Created')
plt.title('Scatter Plot - Logarithmic Scale')
plt.show()

# Set up histogram of the log of total average savings
plt.hist(NTA_over_5['Log Avg Total Savings'])
plt.title('Histogram')
plt.show()

# Create month and year column
df['Effective Date']=pd.to_datetime(df['Effective Date'])
df['month'] = pd.DatetimeIndex(df['Effective Date']).month
df['year'] = pd.DatetimeIndex(df['Effective Date']).year

# Aggregate count of jobs created by month
df_month_agg = df.groupby(['month','year'],as_index=False).agg({'Job created': ['count']}).sort_values(by=['year','month'])
df_month_agg.columns = df_month_agg.columns.get_level_values(0)
df_date_agg = df.groupby(['Effective Date'],as_index=False).agg({'Job created': ['count']}).sort_values(by=['Effective Date'])
df_date_agg.columns = df_date_agg.columns.get_level_values(0)

# Line plot
plt.plot(df_month_agg['month'],df_month_agg['Job created'])
plt.title('Line plot of count of jobs created by month')
plt.xlabel('Month')
plt.ylabel('Jobs Created')
plt.show()

# Month/Year plot
sns.relplot(data=df_month_agg, x="month", y="Job created", hue="year", kind="line")
plt.xlabel('Month')
plt.ylabel('Jobs Created')
plt.title('Line plot - Total jobs created by month')
plt.show()